In [6]:
#scrape from a specified list of channel urls
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By

channel_urls = [
    'https://www.youtube.com/@CNN/videos',
    'https://www.youtube.com/@BBC/videos',
    'https://www.youtube.com/@FoxNews/videos',
    'https://www.youtube.com/@TheMajorityReport/videos'
]

driver_path = r'C:\Users\Dimab\OneDrive\Desktop\Youtube-Web-Scraping\edgedriver_win64\msedgedriver.exe'

option = webdriver.EdgeOptions()
driver = webdriver.Edge(options=option)

video_list = []

for url in channel_urls:
    driver.get(url)

    scroll_pause_time = 1  # Wait for the page to load
    screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web

    i = 1
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after loading more videos
        scroll_height = driver.execute_script("return document.documentElement.scrollHeight;")
        # break the loop if scroll height does not change
        if (screen_height) * i > scroll_height:
            break

    videos = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-rich-item-renderer')

    for video in videos:
        title = video.find_element(By.XPATH, './/*[@id="video-title"]').text
        views = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[1]').text
        when = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[2]').text
        vid_item = {
            'channel': url,
            'title': title,
            'views': views,
            'posted': when
        }

        video_list.append(vid_item)

driver.quit()

df = pd.DataFrame(video_list)
print(df)


                                               channel  \
0                  https://www.youtube.com/@CNN/videos   
1                  https://www.youtube.com/@CNN/videos   
2                  https://www.youtube.com/@CNN/videos   
3                  https://www.youtube.com/@CNN/videos   
4                  https://www.youtube.com/@CNN/videos   
..                                                 ...   
322  https://www.youtube.com/@TheMajorityReport/videos   
323  https://www.youtube.com/@TheMajorityReport/videos   
324  https://www.youtube.com/@TheMajorityReport/videos   
325  https://www.youtube.com/@TheMajorityReport/videos   
326  https://www.youtube.com/@TheMajorityReport/videos   

                                                 title       views  \
0    Actors reject artificial intelligence proposal...  3.8K views   
1          Autopsy reveals why Lisa Marie Presley died   21K views   
2              Putin says Wagner group ‘doesn’t exist'  102K views   
3    Why Defense Secy. 

In [3]:
df.to_csv('video_data.csv', index=False)